<a href="https://colab.research.google.com/github/appletreeleaf/Study_Log/blob/NLP/%5BHW25_Problem%5D_Topic_Modeling_%EC%9D%B4%EC%9E%AC%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [HW2] Topic Modeling
1. Crawling News
2. Preprocessing
3. Build Term-Document Matrix
4. Topic modeling
5. Visualization

```
🔥 이번 시간에는 Topic Modeling를 직접 크롤링한 뉴스 데이터에 대해서 수행해보는 시간을 갖겠습니다.

먼저 네이버에서 뉴스 기사를 간단하게 크롤링합니다.
기본적인 전처리 이후 Term-document Matrix를 만들고 이를 non-negative factorization을 이용해 행렬 분해를 하여 Topic modeling을 수행합니다.

t-distributed stochastic neighbor embedding(T-SNE) 기법을 이용해 Topic별 시각화를 진행합니다.
```

In [ ]:
!pip install newspaper3k
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=2988b289941a8e6e9a832edec7bf0475fcd7443d3a359198d18adf1b18871a95
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=f826e7d1993c5210e8c9b193f2a3c4ed053314ca8975fa5174a1eb

In [ ]:
# 크롤링에 필요한 패키지 설치
from bs4 import BeautifulSoup
from newspaper import Article
from time import sleep
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime
from multiprocessing import Pool
import json
import requests
import re
import sys

```
💡 Crawling(크롤링)이란?

크롤링은 웹 페이지에서 필요한 데이터를 추출해내는 작업을 말합니다.
이번 시간에는 정적 페이지인 네이버의 뉴스 신문 기사 웹페이지를 크롤링합니다.

HTML은 설명되어 있는 자료가 많기 때문에 생략하도록 하겠습니다.
HTML 구조 파악 및 태그에 대한 설명은 아래 참고자료를 살펴봐주세요 !
```

참고: [위키피디아: 정적페이지](https://ko.wikipedia.org/wiki/%EC%A0%95%EC%A0%81_%EC%9B%B9_%ED%8E%98%EC%9D%B4%EC%A7%80)

참고: [생활코딩: HTML](https://opentutorials.org/course/2039)

In [ ]:
def crawl_news(query: str=None, crawl_num: int=1000, workers: int=4):
    '''뉴스 기사 텍스트가 담긴 list를 반환합니다.

    Keyword arguments:
    query -- 검색어 (default None)
    crawl_num -- 수집할 뉴스 기사의 개수 (defualt 1000)
    workers -- multi-processing시 사용할 thread의 개수 (default 4)
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
    articleList = []
    crawled_url = set()
    keyboard_interrupt = False
    t = time()
    idx = 0
    page = 1


    # 서버에 url 요청의 결과를 선언
    res = requests.get(url.format(query))
    sleep(0.5)
    # res를 parsing할 parser를 선언
    bs = BeautifulSoup(res.text, 'html.parser')

    with Pool(workers) as p:
        while idx < crawl_num:
            table = bs.find('ul', {'class': 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class':'news_area'}) for li in li_list]
            a_list = [area.find('a', {'class':'news_tit'}) for area in area_list]

            for n in a_list[:min(len(a_list), crawl_num-idx)]:
                articleList.append(n.get('title'))
                idx += 1
            page += 1

            pages = bs.find('div', {'class': 'sc_page_inner'})
            next_page_url = [p for p in pages.find_all('a') if p.text == str(page)][0].get('href')

            req = requests.get('https://search.naver.com/search.naver' + next_page_url)
            bs = BeautifulSoup(req.text, 'html.parser')
    return articleList

```
🔥 이제 '구글'이라는 이름으로 뉴스 기사 1000개의 제목을 크롤링하겠습니다.
```

In [ ]:
query = '구글'

articleList = crawl_news(query)

In [ ]:
articleList[:10]

['[단독]1년새 200명 늘렸는데…구글코리아 3월 ‘해고 폭풍’ 몰아친다',
 '구글, \'챗GPT 대항마\' 발표…"테스트 거쳐 수주 안에 공개"(종합)',
 "구글, 챗GPT 대항마 '바드'AI 발표...'수주일 내 공개'",
 'MS 검색엔진에 챗GPT 탑재…“구글 잡겠다”',
 "MS, 챗GPT 같은 검색 엔진 '빙' 공개…구글과 한판승부",
 'MS, ‘빙’ 검색에 ‘챗GPT’ 통합···구글과 ‘검색전쟁’',
 "'챗GPT와 바드, MS와 구글'…누가 최종 승자가 될까",
 'MS, 챗GPT 탑재한 ‘검색엔진 빙’ 내놔…구글 심장까지 겨냥',
 '챗GPT 대항마 예고, 구글 가세…시장 선점 경쟁 속 네카오 대응은',
 '구글·모질라, 독자 엔진 기반 iOS용 브라우저 준비']

```
🔥 태거(tagger)를 이용해 한글 명사와 알파벳만을 추출해서 term-document matrix (tdm)을 만들겠습니다.

태거(tagger)는 tokenization에서 조금 더 자세히 다루도록 하겠습니다.
```

참고: [konlpy: morph analyzer](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

## Preprocessing

In [ ]:
from konlpy.tag import Okt
t = Okt()
for i, article in enumerate(articleList):
    if i % 100 == 0:
        print(i)

    # tagger를 이용한 품사 태깅
    words = t.pos(article, norm=True, stem=True)
words

0
100
200
300
400
500
600
700
800
900


[("'", 'Punctuation'),
 ('챗', 'Noun'),
 ('GPT', 'Alpha'),
 ("'", 'Punctuation'),
 ('잡', 'Noun'),
 ('을', 'Josa'),
 ('구글', 'Noun'),
 ('의', 'Josa'),
 ('비밀', 'Noun'),
 ('병기', 'Noun'),
 ('는', 'Josa'),
 ("'", 'Punctuation'),
 ('스패', 'Noun'),
 ('로우', 'Noun'),
 ("'", 'Punctuation')]

In [ ]:
from konlpy.tag import Okt
from collections import Counter
import json

# Okt 형태소 분석기 선언
t = Okt()

words_list_ = []
vocab = Counter()
tag_set = set(['Noun', 'Alpha'])
stopwords = set(['글자'])

for i, article in enumerate(articleList):
    if i % 100 == 0:
        print(i)

    # tagger를 이용한 품사 태깅 / word가 동사, 형용사, 명사 ... 등인지 판단해줌줌
    words = t.pos(article, norm=True, stem=True)

    ############################ ANSWER HERE ################################
    # TODO: 다음의 조건을 만족하는 단어의 리스트를 완성하세요.
    # 조건 1: 명사와 알파벳 tag를 가진 단어
    # 조건 2: 철자 길이가 2이상인 단어
    # 조건 3: stopwords에 포함되지 않는 단어
    words = [w for w,t in words if t in tag_set and len(w)>1 and w not in stopwords]
    #########################################################################

    vocab.update(words) # 사전에 위 조건을 만족하는 word들을 업데이트
    words_list_.append((words, article)) # words와 article제목들을 업데이트트

vocab = sorted([w for w, freq in vocab.most_common(10000)]) # 많이 등장한 단어 순으로 word 정렬
word2id = {w: i for i, w in enumerate(vocab)} # 위 list를 key : word 형태로 변경경
words_list = []
for words, article in words_list_:
    words = [w for w in words if w in word2id]
    if len(words) > 10: #문장길이 10초과
        words_list.append((words, article))

del words_list_

0
100
200
300
400
500
600
700
800
900


In [ ]:
words_list #(기사쪼갠부분, 기사 원문문)

[(['누비', '구글', '순환', '경제', '스타트업', '지원', '프로그램', '국내', '유일', '기업', '선정'],
  '누비랩, 구글 순환 경제 스타트업 지원 프로그램에 국내 유일 기업으로 선정'),
 (['김대호',
   '박사',
   '오늘',
   '기업',
   '사람',
   '바이두',
   '구글',
   'MS',
   '라피',
   '더스',
   '삼성',
   '전자',
   'OCI',
   'SGC',
   '에너지'],
  '[김대호 박사의 오늘 기업·사람] 바이두·구글·MS·라피더스·삼성전자·OCI·SGC에너지'),
 (['특징', '미디어', '구글', '애플', '협업', '이력', '챗봇', 'AI', '원천', '기술', '대감', '상승세'],
  '[특징주] 미디어젠, 구글-애플 협업 이력...챗봇 등 AI원천 기술 기대감에 상승세'),
 (['박성', '네이버', '뉴스', '아웃', '링크', '제공', '필요', '구글', '바이두', '아웃', '링크', '실시'],
  '박성중 "네이버, 뉴스 \'아웃링크\' 제공 필요…구글·바이두, 아웃링크 실시"'),
 (['김대호',
   '박사',
   '오늘',
   '기업',
   '사람',
   '구글',
   'MS',
   '네이버',
   '테슬라',
   '현대차',
   'CJ',
   '투썸플레이스'],
  '[김대호 박사의 오늘 기업·사람] 구글·MS·네이버·테슬라·현대차·CJ·투썸플레이스'),
 (['삼성',
   '전자',
   '퀄컴',
   '구글',
   'XR',
   '동맹',
   '결의',
   '메타',
   '애플',
   '메타',
   '버스',
   '삼국시대',
   '예고'],
  '삼성전자, 퀄컴·구글과 ‘XR 동맹’ 결의… 메타·애플과 ‘메타버스 삼국시대’ 예고'),
 (['이항', '가이드', '알파벳', '구글', '바드', 'Bard', 'AI', '발표', '대감', '주가', '흐름'],
  "

## Build document-term matrix

```
🔥 이제 document-term matrix를 만들어보겠습니다.
document-term matrix는 (문서 개수 x 단어 개수)의 Matrix입니다.
즉 (doc * word)
```

참고: [Document-Term Matrix](https://wikidocs.net/24559)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer # TF-IDF score
import numpy as np

dtm = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # (doc * words)
for i, (words, article) in enumerate(words_list):
    for word in words:
        dtm[i, word2id[word]] += 1

dtm = TfidfTransformer().fit_transform(dtm)
print(dtm)

  (0, 1299)	0.32887493312543237
  (0, 1090)	0.30182625403297253
  (0, 884)	0.32887493312543237
  (0, 699)	0.32887493312543237
  (0, 694)	0.32887493312543237
  (0, 647)	0.3142182809921697
  (0, 276)	0.32887493312543237
  (0, 229)	0.25206228641086603
  (0, 200)	0.30182625403297253
  (0, 193)	0.08112318153865841
  (0, 145)	0.32887493312543237
  (1, 1005)	0.23911076813977752
  (1, 833)	0.2460113314696681
  (1, 792)	0.31235178271682174
  (1, 611)	0.20345493460973785
  (1, 591)	0.253639547257554
  (1, 470)	0.253639547257554
  (1, 467)	0.2621671700690056
  (1, 378)	0.31235178271682174
  (1, 335)	0.31235178271682174
  (1, 238)	0.253639547257554
  (1, 229)	0.2270158634176082
  (1, 193)	0.0730622948891414
  (1, 50)	0.31235178271682174
  (1, 46)	0.31235178271682174
  :	:
  (106, 126)	0.288911383221758
  (107, 1308)	0.29786982575213344
  (107, 1139)	0.29786982575213344
  (107, 1060)	0.31923822713430494
  (107, 830)	0.31923822713430494
  (107, 601)	0.31923822713430494
  (107, 564)	0.297869825752133

```
🔥 document-term matrix를 non-negative factorization(NMF)을 이용해 행렬 분해를 해보겠습니다.

💡 Non-negative Factorization이란?

NMF는 주어진 행렬 non-negative matrix X를 non-negative matrix W와 H로 행렬 분해하는 알고리즘입니다.
이어지는 코드를 통해 W와 H의 의미에 대해 파악해봅시다.
```
참고: [Non-negative Matrix Factorization](https://angeloyeo.github.io/2020/10/15/NMF.html)

## Topic modeling

In [ ]:
# Non-negative Matrix Factorization
from sklearn.decomposition import NMF

K=5             # topic의 수
nmf = NMF(n_components=K, alpha=0.1)

```
🔥 sklearn의 NMF를 이용해 W와 H matrix를 구해봅시다.
W는 document length x K, H는 K x term length의 차원을 갖고 있습니다.
W의 하나의 row는 각각의 feature에 얼만큼의 가중치를 줄 지에 대한 weight입니다.
H의 하나의 row는 하나의 feature를 나타냅니다.

우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [ ]:
W = nmf.fit_transform(dtm)
H = nmf.components_

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


```
🔥 우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [ ]:
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
    print()

0th topic
AI GPT 챗봇 바드 특징 구글 출시 대항 MS 급등 관련 발표 엔젤 소식 공식 상하 강세 파트너 경쟁 테크 
1th topic
하나 함영주 방문 금융 회장 혁신 엔비디아 체험 디지털 기술 구글 간다 도전 감히 위해 선도 인사이트 본사 DNA 확장 
2th topic
버디 게임 체인 블록 골프 글로벌 플레이 주얼 돌입 사전 예약 사전예약 보라 메타 구글 시작 카카오 스토어 구글플레이 프렌즈 
3th topic
기업 박사 사람 김대호 오늘 MS 현대차 투썸플레이스 CJ 테슬라 네이버 에너지 더스 OCI 라피 SGC 바이두 구글 전자 삼성 
4th topic
삼성 XR 메타 퀄컴 동맹 버스 전자 애플 삼국시대 결의 콘텍 신화 구축 구글 예고 깜짝 맥스 부각 발표 개발 


```
🔥 이번에는 W에서 하나의 Topic (W의 n번째 column)에 접근해서 해당 topic에 대해 값이 가장 높은 3개의 뉴스 기사 제목을 출력해보겠습니다.
```

In [ ]:
for k in range(K):
    print(f"==={k}th topic===")
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1])
    print('\n')

===0th topic===
[특징주] 유엔젤, 구글 챗GPT 대항마 '바드'AI 발표... AI 챗봇 서비스 구글 솔루션 연동 부각
[7일 특징주] 구글, 챗GPT 대항마 '바드' 출시 소식에 관련주 급등…유엔젤 등 6개 종목 상한가
[7일 특징주] 구글, 챗GPT 대항마 '바드' 출시 소식에 관련주 급등…유엔젤 등 6개 종목 상한가


===1th topic===
함영주 하나금융 회장, 구글·엔비디아 방문…디지털 혁신 기술 체험
함영주 하나금융 회장, 구글·엔비디아 방문…디지털 혁신 기술 체험
함영주 하나금융 회장, 구글·엔비디아 방문…디지털 혁신 기술 체험


===2th topic===
블록체인 캐주얼 골프 게임 ‘버디샷’, 구글 플레이 글로벌 사전 예약 돌입
블록체인 캐주얼 골프 게임 ‘버디샷’ 구글 플레이 글로벌 사전 예약 돌입
블록체인 캐주얼 골프 게임 ‘버디샷’, 구글 플레이 글로벌 사전 예약 돌입


===3th topic===
[김대호 박사의 오늘 기업·사람] 구글·MS·네이버·테슬라·현대차·CJ·투썸플레이스
[김대호 박사의 오늘 기업·사람] 구글·MS·네이버·테슬라·현대차·CJ·투썸플레이스
[김대호 박사의 오늘 기업·사람] 구글·MS·네이버·테슬라·현대차·CJ·투썸플레이스


===4th topic===
삼성전자, 퀄컴·구글과 ‘XR 동맹’ 결의… 메타·애플과 ‘메타버스 삼국시대’ 예고
삼성전자, 퀄컴·구글과 ‘XR 동맹’ 결의… 메타·애플과 ‘메타버스 삼국시대’ 예고
[ET라씨로] 삼성전자, 구글·퀄컴과 메타버스 동맹 구축…엔피 부각




```
❓ 2번째 토픽에 대해 가장 높은 가중치를 갖는 제목 5개를 출력해볼까요?
```

In [ ]:
print(f"==={2}th topic===")
for index in W[:, 2].argsort()[::-1][:5]:
  print(words_list[index][1])

===2th topic===
블록체인 캐주얼 골프 게임 ‘버디샷’, 구글 플레이 글로벌 사전 예약 돌입
블록체인 캐주얼 골프 게임 ‘버디샷’ 구글 플레이 글로벌 사전 예약 돌입
블록체인 캐주얼 골프 게임 ‘버디샷’, 구글 플레이 글로벌 사전 예약 돌입
블록체인 캐주얼 골프 게임 ‘버디샷’. 구글 플레이 스토어 글로벌 사전 예약 돌입
블록체인 캐주얼 골프 게임 '버디샷', 구글 플레이 글로벌 사전예약 돌입


```
🔥 이번에는 t-SNE를 이용해 Topic별 시각화를 진행해보겠습니다.

💡 t-SNE는 무엇인가요?

t-Stochastic Neighbor Embedding(t-SNE)은 고차원의 벡터를
저차원(2~3차원) 벡터로 데이터간 구조적 특징을 유지하며 축소를 하는 방법 중 하나입니다.

주로 고차원 데이터의 시각화를 위해 사용됩니다.
```

참고: [lovit: t-SNE](https://lovit.github.io/nlp/representation/2018/09/28/tsne/#:~:text=t%2DSNE%20%EB%8A%94%20%EA%B3%A0%EC%B0%A8%EC%9B%90%EC%9D%98,%EC%9D%98%20%EC%A7%80%EB%8F%84%EB%A1%9C%20%ED%91%9C%ED%98%84%ED%95%A9%EB%8B%88%EB%8B%A4.)

참고: [ratsgo: t-SNE](https://ratsgo.github.io/machine%20learning/2017/04/28/tSNE/)

## Visualization

In [ ]:
from sklearn.manifold import TSNE

# n_components = 차원 수
tsne = TSNE(n_components=2, init='pca', verbose=1) #pca를 활용한 demension reduction

# W matrix에 대해 t-sne를 수행합니다.
W2d = tsne.fit_transform(W)

# 각 뉴스 기사 제목마다 가중치가 가장 높은 topic을 저장합니다.
topicIndex = [v.argmax() for v in W]

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 109 samples in 0.001s...
[t-SNE] Computed neighbors for 109 samples in 0.012s...
[t-SNE] Computed conditional probabilities for sample 109 / 109
[t-SNE] Mean sigma: 0.099171
[t-SNE] KL divergence after 250 iterations with early exaggeration: 52.477661
[t-SNE] KL divergence after 1000 iterations: -0.215956


In [ ]:
print(W2d)

[[  21.189995    133.97551   ]
 [ 153.6233      -80.72063   ]
 [ -73.42428     -50.7655    ]
 [  80.49761     168.5521    ]
 [ 119.565735   -103.307976  ]
 [ 364.16806     -59.94793   ]
 [ -98.21515      28.927088  ]
 [-281.03226     -80.29941   ]
 [-146.91231     -19.646923  ]
 [ -65.00991      -2.2413487 ]
 [ -48.36428      39.361877  ]
 [-320.68796       8.545354  ]
 [  -4.411863     34.800735  ]
 [-167.27925    -101.8205    ]
 [-335.6648      -38.829163  ]
 [-294.6967      -34.847103  ]
 [  21.189995    133.97551   ]
 [-257.2159      -38.603405  ]
 [  21.189995    133.97551   ]
 [ -78.321144     68.31315   ]
 [-257.2159      -38.603405  ]
 [-178.06487     -38.60738   ]
 [  82.539825    -90.82801   ]
 [-248.51967      26.507471  ]
 [ 123.90938     -62.726864  ]
 [   4.084484    -20.702345  ]
 [ -23.663826      3.1789725 ]
 [-191.35352       2.174387  ]
 [ -73.42428     -50.7655    ]
 [-234.3472      -86.01091   ]
 [  80.49761     168.5521    ]
 [ 119.565735   -103.307976  ]
 [ 119.5

In [ ]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article for words, article in words_list],
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color'
)

# interaction
p.legend.location = "top_left"
hover = p.select({'type': HoverTool})
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)